In [1]:
import cirq
import cirq_google as cg
import torch
import numpy as np
from shallow_cirq1 import get_circuit, get_two_qubits, get_single_qubit

/Users/hwanda/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Set up the real engine and sampler on Willow Delphi
run_name = 'Feb_1_cz_calibration'
eng = cg.get_engine('serious-cat-404923')
sampler_real = eng.get_sampler(processor_id='WIL01_3A_DELPHI', run_name=run_name, device_config_name='cz_71')

In [3]:
# Set up simulate sampler
sampler_sim = cirq.Simulator()

In [3]:
# test Bell state
qubits = [cirq.GridQubit(4,4), cirq.GridQubit(4,5)]
cir = cirq.Circuit(cirq.H.on_each(*qubits), cirq.CZ(*qubits), cirq.H(qubits[1]), cirq.M(*qubits, key='m'))
result = sampler_real.run(cir, repetitions=1000)
result.histogram(key='m')

Counter({0: 497, 3: 485, 1: 6, 2: 12})

### Two probe qubits experiment

#### Get qubits

In [4]:
#size = '3x3'
#size = '5x5'
size = '6x6'
qubits_matrix, probe_qubits, anc_pairs, all_qubits = get_two_qubits(size, anc=True)
print(len(all_qubits))
# circ = get_circuit(qubits_matrix, theta=1, phi=1, probe_qubits=probe_qubits, basis=[1,1], anc_pairs=anc_pairs)
# circ

55


In [5]:
probe_qubits

[cirq.GridQubit(3, 7), cirq.GridQubit(8, 7)]

#### Collect data

In [6]:
# collect data
sampler = sampler_real
theta_range = np.linspace(0, np.pi/2, 11)
phi = np.pi*(5/4)
repetitions = 10**5

for i in range(1):
    for theta_idx in [10]:
        for b1 in range(3):
            for b2 in range(3):
                circ = get_circuit(qubits_matrix, theta=theta_range[theta_idx], phi=phi, probe_qubits=probe_qubits, basis=[b1,b2], anc_pairs=anc_pairs)
                circ.append(cirq.measure(*all_qubits, key='m'))
                result = sampler.run(circ, repetitions=repetitions)
                hist = result.histogram(key='m')
                print(f'loop={i}, size={size}, theta={theta_idx}, basis={b1},{b2}')
                np.save(f'data/{run_name}_d=5/two_probes_{i}/data_{size}/hist/theta={theta_idx}_({b1},{b2}).npy', np.array(list(hist.items()))) # save histogram
                torch.save(torch.tensor(result.measurements['m']).to(int), f'data/{run_name}_d=5/two_probes_{i}/data_{size}/seq/theta={theta_idx}_({b1},{b2}).pt') # save sequence

loop=0, size=6x6, theta=10, basis=0,0
loop=0, size=6x6, theta=10, basis=0,1
loop=0, size=6x6, theta=10, basis=0,2
loop=0, size=6x6, theta=10, basis=1,0
loop=0, size=6x6, theta=10, basis=1,1
loop=0, size=6x6, theta=10, basis=1,2
loop=0, size=6x6, theta=10, basis=2,0
loop=0, size=6x6, theta=10, basis=2,1
loop=0, size=6x6, theta=10, basis=2,2


In [7]:
result.measurements['m'].shape

(100000, 55)

In [ ]:
print("Circuit depth:", len(circ))
#print(circ)

In [ ]:
all_qubits

In [ ]:
print(all_qubits[0], all_qubits[2], all_qubits[8])
print(all_qubits[-1], all_qubits[-4], all_qubits[-9])

In [ ]:
print((result.measurements['m'][:,-1]==result.measurements['m'][:,-9]).mean())


### Single qubit experiment

#### Get qubits

In [7]:
#size = '3x3'
#size = '5x5'
size = '6x6'
qubits_matrix, probe_qubits, anc_pairs, all_qubits = get_single_qubit(size, anc=True)

#### Collect data

In [8]:
# collect data
sampler = sampler_real
theta_range = np.linspace(0, np.pi/2, 11)
phi = np.pi*(5/4)
repetitions = 10**6

for i in range(5,10): # loop over 5 times
    for theta_idx in range(11):
        for b in range(3):
            circ = get_circuit(qubits_matrix, theta=theta_range[theta_idx], phi=phi, probe_qubits=probe_qubits, basis=[b], anc_pairs=anc_pairs)
            circ.append(cirq.measure(*all_qubits, key='m'))
            result = sampler.run(circ, repetitions=repetitions)
            hist = result.histogram(key='m')
            print(f'loop={i}, size={size}, theta={theta_idx}, basis={b}')
            np.save(f'data/{run_name}/single_probe_{i}/data_{size}/hist/theta={theta_idx}_({b}).npy', np.array(list(hist.items()))) # save histogram
            torch.save(torch.tensor(result.measurements['m']).to(int), f'data/{run_name}/single_probe_{i}/data_{size}/seq/theta={theta_idx}_({b}).pt') # save sequence

loop=5, size=6x6, theta=0, basis=0
loop=5, size=6x6, theta=0, basis=1
loop=5, size=6x6, theta=0, basis=2
loop=5, size=6x6, theta=1, basis=0
loop=5, size=6x6, theta=1, basis=1
loop=5, size=6x6, theta=1, basis=2
loop=5, size=6x6, theta=2, basis=0
loop=5, size=6x6, theta=2, basis=1
loop=5, size=6x6, theta=2, basis=2
loop=5, size=6x6, theta=3, basis=0
loop=5, size=6x6, theta=3, basis=1
loop=5, size=6x6, theta=3, basis=2
loop=5, size=6x6, theta=4, basis=0
loop=5, size=6x6, theta=4, basis=1
loop=5, size=6x6, theta=4, basis=2
loop=5, size=6x6, theta=5, basis=0
loop=5, size=6x6, theta=5, basis=1
loop=5, size=6x6, theta=5, basis=2
loop=5, size=6x6, theta=6, basis=0
loop=5, size=6x6, theta=6, basis=1
loop=5, size=6x6, theta=6, basis=2
loop=5, size=6x6, theta=7, basis=0
loop=5, size=6x6, theta=7, basis=1
loop=5, size=6x6, theta=7, basis=2
loop=5, size=6x6, theta=8, basis=0
loop=5, size=6x6, theta=8, basis=1
loop=5, size=6x6, theta=8, basis=2
loop=5, size=6x6, theta=9, basis=0
loop=5, size=6x6, th

In [7]:
result.measurements['m'].shape

(1000000, 55)

In [ ]:
print("Circuit depth:", len(circ))

In [ ]:
all_qubits

In [ ]:
print(all_qubits[4], all_qubits[10], all_qubits[11])

In [ ]:
print((result.measurements['m'][:,4]==result.measurements['m'][:,11]).mean())


In [ ]:
9*10**6/2**23